In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import shutil
import operator

# ===================================================================
# 설정 (사용자 환경에 맞게 경로를 수정해주세요)
# ===================================================================
# 1. 학습된 모델 파일 경로
model_path = "C:\\Users\\it\\Documents\\Python-workspace\\Ai-ML-DL-workspace-0930\\memo\\3-Resnet\\test\\best_model.pth"

# 2. 검수할 원본 이미지 데이터가 있는 폴더 경로
data_dir = "C:\\Users\\it\\Documents\\Python-workspace\\Ai-ML-DL-workspace-0930\\memo\\3-Resnet\\test\\class\\"
# ===================================================================

print("🚀 1단계: 모델 및 데이터 경로 설정을 시작합니다...")

class_names = sorted([d.name for d in os.scandir(data_dir) if d.is_dir()])
num_classes = len(class_names)

model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

transform_pred = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print(f"✅ 1단계 완료: 모델 및 데이터 준비 완료. 클래스: {class_names}")

🚀 1단계: 모델 및 데이터 경로 설정을 시작합니다...
✅ 1단계 완료: 모델 및 데이터 준비 완료. 클래스: ['의자_images', '책상_images']


In [31]:
print("🚀 2단계: 전체 이미지에 대한 예측을 시작합니다... (시간 소요)")

results = []
image_paths = []
for class_index, class_name in enumerate(class_names):
    class_dir_path = os.path.join(data_dir, class_name)
    for fname in os.listdir(class_dir_path):
        fpath = os.path.join(class_dir_path, fname)
        try:
            Image.open(fpath).verify()
            image_paths.append((fpath, class_index))
        except Exception:
            print(f"⚠️ 이미지가 아니거나 손상된 파일은 건너뜁니다: {fpath}")

with torch.no_grad():
    for i, (path, true_label_index) in enumerate(image_paths):
        if (i + 1) % 50 == 0:
            print(f"  - 진행 중: {i + 1} / {len(image_paths)}")
        try:
            image = Image.open(path).convert("RGB")
            image_tensor = transform_pred(image).unsqueeze(0)
            output = model(image_tensor)
            probabilities = F.softmax(output, dim=1)[0]
            predicted_label_index = torch.argmax(probabilities).item()
            confidence = probabilities[predicted_label_index].item()
            all_probs = probabilities.cpu().numpy()
            results.append({
                "path": path, "true_label": class_names[true_label_index],
                "predicted_label": class_names[predicted_label_index],
                "confidence": confidence, "is_correct": true_label_index == predicted_label_index,
                "probabilities": all_probs
            })
        except Exception as e:
            print(f"❌ 예측 중 오류 발생: {path} - {e}")

print(f"✅ 2단계 완료: 총 {len(results)}개 이미지 예측 완료!")

🚀 2단계: 전체 이미지에 대한 예측을 시작합니다... (시간 소요)
⚠️ 이미지가 아니거나 손상된 파일은 건너뜁니다: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-Resnet\test\class\책상_images\책상_72.jpg
  - 진행 중: 50 / 493
  - 진행 중: 100 / 493
  - 진행 중: 150 / 493
  - 진행 중: 200 / 493
  - 진행 중: 250 / 493
  - 진행 중: 300 / 493
  - 진행 중: 350 / 493
  - 진행 중: 400 / 493
  - 진행 중: 450 / 493
✅ 2단계 완료: 총 493개 이미지 예측 완료!


In [32]:
print("🚀 3단계: 예측 결과를 바탕으로 '문제 데이터'를 분류합니다...")

misclassified_confident = [res for res in results if not res["is_correct"] and res["confidence"] > 0.90]
ambiguous_cases = [res for res in results if 0.5 <= res["confidence"] < 0.65]

for res in results:
    true_label_idx = class_names.index(res['true_label'])
    res['loss'] = -torch.log(torch.tensor(res['probabilities'][true_label_idx]) + 1e-9).item()
hardest_cases = sorted(results, key=operator.itemgetter('loss'), reverse=True)

print("✅ 3단계 완료: 문제 데이터 분류 완료.")
print(f"  - 라벨링 오류 의심: {len(misclassified_confident)}개")
print(f"  - 애매한 데이터 의심: {len(ambiguous_cases)}개")
print(f"  - 가장 어려운 데이터 (Top 20): {min(20, len(hardest_cases))}개")

🚀 3단계: 예측 결과를 바탕으로 '문제 데이터'를 분류합니다...
✅ 3단계 완료: 문제 데이터 분류 완료.
  - 라벨링 오류 의심: 2개
  - 애매한 데이터 의심: 126개
  - 가장 어려운 데이터 (Top 20): 20개


In [33]:
print("🚀 4단계: 검토용 폴더를 생성하고 의심스러운 파일들을 '이동'합니다...")

review_base_dir = os.path.abspath(os.path.join(data_dir, "..", "review_folders"))
if os.path.exists(review_base_dir):
    shutil.rmtree(review_base_dir)
os.makedirs(review_base_dir, exist_ok=True)
print(f"📂 검토용 상위 폴더 생성: {review_base_dir}")

moved_files = set()

for class_name in class_names:
    class_review_dir = os.path.join(review_base_dir, class_name)
    os.makedirs(class_review_dir, exist_ok=True)
    print(f"\n[클래스: {class_name}] 검토 시작...")

    # 유형 1: 라벨링 오류 의심
    dir_label_error = os.path.join(class_review_dir, "1_labeling_error_suspicion")
    os.makedirs(dir_label_error, exist_ok=True)
    files_to_move = [res for res in misclassified_confident if res['true_label'] == class_name]
    moved_count = 0
    for res in files_to_move:
        if res["path"] in moved_files: continue
        fname = os.path.basename(res["path"])
        shutil.move(res["path"], os.path.join(dir_label_error, fname))  # 원본 파일명 그대로 이동
        moved_files.add(res["path"])
        moved_count += 1
    print(f"  - 유형 1 (라벨링 오류 의심): {moved_count}개 파일 이동 완료.")

    # 유형 2: 애매한 데이터 의심
    dir_ambiguous = os.path.join(class_review_dir, "2_ambiguous_data_suspicion")
    os.makedirs(dir_ambiguous, exist_ok=True)
    files_to_move = [res for res in ambiguous_cases if res['true_label'] == class_name]
    moved_count = 0
    for res in files_to_move:
        if res["path"] in moved_files: continue
        fname = os.path.basename(res["path"])
        shutil.move(res["path"], os.path.join(dir_ambiguous, fname))  # 원본 파일명 그대로 이동
        moved_files.add(res["path"])
        moved_count += 1
    print(f"  - 유형 2 (애매한 데이터 의심): {moved_count}개 파일 이동 완료.")

    # 유형 3: 모델이 가장 어려워하는 데이터
    dir_hardest = os.path.join(class_review_dir, "3_hardest_to_learn")
    os.makedirs(dir_hardest, exist_ok=True)
    files_to_move = [res for res in hardest_cases if res['true_label'] == class_name][:20]
    moved_count = 0
    for res in files_to_move:
        if res["path"] in moved_files: continue
        fname = os.path.basename(res["path"])
        shutil.move(res["path"], os.path.join(dir_hardest, fname))  # 원본 파일명 그대로 이동
        moved_files.add(res["path"])
        moved_count += 1
    print(f"  - 유형 3 (가장 어려운 데이터 Top 20): {moved_count}개 파일 이동 완료.")

print(f"\n\n✅ 모든 의심 파일을 'review_folders'로 이동했습니다. 이제 직접 폴더를 확인하고, 잘못된 파일을 삭제해주세요.")

🚀 4단계: 검토용 폴더를 생성하고 의심스러운 파일들을 '이동'합니다...
📂 검토용 상위 폴더 생성: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-Resnet\test\review_folders

[클래스: 의자_images] 검토 시작...
  - 유형 1 (라벨링 오류 의심): 1개 파일 이동 완료.
  - 유형 2 (애매한 데이터 의심): 54개 파일 이동 완료.
  - 유형 3 (가장 어려운 데이터 Top 20): 15개 파일 이동 완료.

[클래스: 책상_images] 검토 시작...
  - 유형 1 (라벨링 오류 의심): 1개 파일 이동 완료.
  - 유형 2 (애매한 데이터 의심): 72개 파일 이동 완료.
  - 유형 3 (가장 어려운 데이터 Top 20): 7개 파일 이동 완료.


✅ 모든 의심 파일을 'review_folders'로 이동했습니다. 이제 직접 폴더를 확인하고, 잘못된 파일을 삭제해주세요.


In [35]:
print("🚀 5단계: 검토 완료된 깨끗한 파일들을 원본 위치로 복원합니다...")
print("          ('review_folders'에서 삭제하지 않고 남겨둔 파일들이 대상입니다)")

review_base_dir = os.path.abspath(os.path.join(data_dir, "..", "review_folders"))

if not os.path.exists(review_base_dir):
    print("⚠️ 'review_folders'가 없습니다. 복원할 파일이 없습니다.")
else:
    restored_count = 0
    for class_name in os.listdir(review_base_dir):
        class_review_dir = os.path.join(review_base_dir, class_name)
        if not os.path.isdir(class_review_dir): continue

        original_destination_dir = os.path.join(data_dir, class_name)
        if not os.path.exists(original_destination_dir):
            os.makedirs(original_destination_dir)

        for suspicion_type_folder in os.listdir(class_review_dir):
            suspicion_dir_path = os.path.join(class_review_dir, suspicion_type_folder)
            if not os.path.isdir(suspicion_dir_path): continue

            for fname in os.listdir(suspicion_dir_path):
                current_path = os.path.join(suspicion_dir_path, fname)
                # 원본 파일명이 유지되었으므로, 바로 목적지 경로로 이동
                destination_path = os.path.join(original_destination_dir, fname)

                shutil.move(current_path, destination_path)
                restored_count += 1

    print(f"\n✅ 5단계 완료: 총 {restored_count}개의 파일을 원본 위치로 복원했습니다.")

    try:
        shutil.rmtree(review_base_dir)
        print("🗑️ 'review_folders'를 성공적으로 삭제했습니다.")
    except OSError as e:
        print(f"🗑️ 'review_folders' 삭제 중 오류 발생 (폴더가 비어있지 않을 수 있음): {e}")

🚀 5단계: 검토 완료된 깨끗한 파일들을 원본 위치로 복원합니다...
          ('review_folders'에서 삭제하지 않고 남겨둔 파일들이 대상입니다)

✅ 5단계 완료: 총 22개의 파일을 원본 위치로 복원했습니다.
🗑️ 'review_folders'를 성공적으로 삭제했습니다.
